# Backend Database (Part 2: SeatGeek)

The foundation of our backend is to have an extensive list of artist and information relevant to them. We will use X different APIs to collect various information:

1. lastfm API to gather a long list of artists, mainly those popular in the US.
2. **SeatGeek API to gather data on upcoming concerts, particularly ticket pricing and concert size.**

In [3]:
import pandas as pd
import requests
from datetime import datetime
from pytz import timezone
import re
import MySQLdb as mdb
import sys
import pickle

## Fetch a list of artist names from a pickled list (originally lastfm data)

The idea here is that we do not want to overload the API with requests (the number of artists is 9029 artists), which is likely to result in a `429 Too Many Requests` status code. Therefore, we will create the input like this:

1. Read a pickled file containing all the artist names, already formatted to plug into API
2. Index only [0:20] of the list, so we limit our queries to 20 names at a time.
3. Shift the entire list by 20 units by defining a `shift` function.
4. Save the shifted list by pickling it.
5. Repeat!

This helps to avoid a 429 error and allows us to run the program on crontab without having to manually change the artists being queried.

In [4]:
#A function that shifts the list each time
def shift(pickled_list, n):
    return pickled_list[n:] + pickled_list[:n]

In [5]:
#Open pickled file containing the artist names
fileObject = open("/home/ubuntu/jupyter/Student_Notebooks/Assignments/Project/Crontab/seatgeek_artist_name_list",'rb') 
old_pickled_list = pickle.load(fileObject)

In [14]:
#Restricting the search to 20 artists 
list_20 = old_pickled_list[0:20]

## Upcoming Concerts Information

In [10]:
#Function for Table 1. This gets a list of upcoming events and related information
def get_event_data_seatgeek(artist):
    ''' Fetches upcoming concert/events information'''
    url = "https://api.seatgeek.com/2/events?performers.slug={}&client_id=<clientid>".format(artist)
    try:
        if requests.get(url).json()['events'] != None:
            events = requests.get(url).json()['events']
            events_list = []
            for i in range(len(events)-1):
                dict_event = {}
                dict_event['artist'] = artist
                dict_event['title'] = events[i]['short_title'] #title
                dict_event['concert_date'] = datetime.strptime(events[i]['datetime_utc'][0:10], '%Y-%m-%d') #datetime of performance
                dict_event['address'] = events[0]['venue']['address'] + ', ' + events[0]['venue']['extended_address'] #address
                dict_event['venue'] = events[0]['venue']['name'].lower().replace(" ", "-")#venue name
                dict_event['lat'] = events[i]['venue']['location']['lat'] #latitude
                dict_event['lon'] = events[i]['venue']['location']['lon'] #longitude
                dict_event['average_price'] = events[i]['stats']['average_price'] #average price
                dict_event['highest_price'] = events[i]['stats']['highest_price'] #highest price
                dict_event['lowest_price'] = events[i]['stats']['lowest_price'] #lowest price
                dict_event['good_deals'] = events[i]['stats']['lowest_price_good_deals'] #best deal
                events_list.append(dict_event)
            return events_list
        else:
            None
    except:
        None

In [12]:
#For loop to get all the results in one list
input_list = []
for name in list_20:
    input_list.append(get_event_data_seatgeek(name))    

In [13]:
#Get rid of artists that have no upcoming concerts, or are not on the Seatgeek database
input_list = [x for x in input_list if x != None] 
input_list = [x for x in input_list if x != []]

In [15]:
#Connecting to MySQL database
con = mdb.connect(host = 'localhost', 
                  user = 'root',
                  passwd = '<password>', 
                  charset='utf8', use_unicode=True);

In [16]:
#Create a table for SeatGeek Concerts
cursor = con.cursor()
table_name = 'seatgeek_concerts'

create_table_query = '''CREATE TABLE IF NOT EXISTS {db}.{table} 
                                (artist varchar(250), 
                                concert_date datetime,
                                average_price int,
                                highest_price int,
                                lowest_price int,
                                good_deals int,
                                address varchar(250),
                                venue varchar(250),
                                latitude int,
                                longitude int,
                                update_time datetime,
                                PRIMARY KEY(artist, concert_date)
                                )'''.format(db='Project', table=table_name)
cursor.execute(create_table_query)
cursor.close()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:19: Warning: (1050, "Table 'seatgeek_concerts' already exists")


In [17]:
#Insert concert data
cursor = con.cursor()
table_name = 'seatgeek_concerts'

query_template = '''INSERT IGNORE INTO {db}.{table}(artist, 
                                            concert_date,
                                            average_price,
                                            highest_price,
                                            lowest_price,
                                            good_deals,
                                            address,
                                            venue,
                                            latitude,
                                            longitude,
                                            update_time) 
                                            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'''.format(db="Project", table=table_name)

cursor = con.cursor()



for data in input_list:
    if data != None:
        for j in range(len(data)):
            artist = data[j]['artist']
            concert_date = data[j]['concert_date']
            average_price = data[j]['average_price']
            highest_price = data[j]['highest_price']
            lowest_price = data[j]['lowest_price']
            good_deals = data[j]['good_deals']
            address = data[j]['address']
            venue = data[j]['venue']
            latitude = data[j]['lat']
            longitude = data[j]['lon']
            update_time = datetime.now(timezone('America/New_York')).strftime("%Y-%m-%d %H:%M:%S %Z%z")
            
            query_parameters = (artist, concert_date, average_price, highest_price,
                            lowest_price, good_deals, address, venue,
                            latitude, longitude, update_time)
            cursor.execute(query_template, query_parameters)

con.commit()
cursor.close()

## B) Artists' Information

In [ ]:
#Functions for Table 2. This gets SeatGeek information on each artist.
def get_genres(info_artist):
    ''' Function to get the genres'''
    string = ""
    genres = info_artist['performers'][0]['genres']
    for i in range(len(genres)):
        string += genres[i]['slug'] + "/"
    return string

def get_artist_data_seatgeek(artist):
    ''' Function to get SeatGeek data on each artist. Quite limited but could be useful'''
    url = "https://api.seatgeek.com/2/performers?slug=" + artist + "&client_id=<clientid>"
    try:
        if requests.get(url).json() != None:
            info_artist = requests.get(url).json()
            infolist = {}
            infolist['name'] = artist
            infolist['popularity'] = info_artist['performers'][0]['score']
            infolist['genres'] = get_genres(info_artist)
        return infolist
    except:
        None

In [ ]:
#Connecting to MySQL database
con = mdb.connect(host = 'localhost', 
                  user = 'root',
                  passwd = '<password>', 
                  charset='utf8', use_unicode=True);

In [ ]:
#Create a table for SeatGeek Artists
cursor = con.cursor()
table_name = 'seatgeek_artists'

create_table_query = '''CREATE TABLE IF NOT EXISTS {db}.{table} 
                                (artist varchar(250), 
                                popularity varchar(250),
                                genres varchar(250),
                                PRIMARY KEY(artist)
                                )'''.format(db='Project', table=table_name)
cursor.execute(create_table_query)
cursor.close()

In [ ]:
#Insert artist data
cursor = con.cursor()
table_name = 'seatgeek_artists'

query_template = '''INSERT IGNORE INTO {db}.{table}(artist, 
                                            popularity,
                                            genres) 
                                            VALUES (%s, %s, %s)'''.format(db="Project", table=table_name)

cursor = con.cursor()


for name in list_20:
    dict1 = get_artist_data_seatgeek(name)
    if dict1 != None:
        artist = name
        popularity = dict1['popularity']
        genres = dict1['genres']
    
        query_parameters = (artist, popularity, genres)
        cursor.execute(query_template, query_parameters)

con.commit()
cursor.close()

## Shift the entire list and pickle the new list to replace file

In [17]:
shifted_list_names = shift(old_pickled_list, 20)

In [ ]:
#Updating pickle file with current artists
file_Name = "/home/ubuntu/jupyter/Student_Notebooks/Assignments/Project/Crontab/seatgeek_artist_name_list"
fileObject = open(file_Name,'wb') 

pickle.dump(shifted_list_names,fileObject)
fileObject.close()